In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [2]:
MODEL = "gpt-3.5-turbo"
OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"
S2W = "../data/s2w_sample_improved.csv"
S2W_SUMMARIZED = "../data/s2w_summarized.csv"
S2W_SAMPLE = "../data/s2w_sample.csv"

In [3]:
s2w = pd.read_csv(S2W)
s2w.drop(columns=["Unnamed: 0"], inplace=True)

s2w_summarized = pd.read_csv(S2W_SUMMARIZED)
s2w_summarized.drop(columns=["Unnamed: 0"], inplace=True)

s2w_sample = pd.read_csv(S2W_SAMPLE)
s2w_sample.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
from notebooks.prompts import CREATION_PROMPT_FS, CREATION_PROMPT_ZS

def create_GUI(id, prompt_type="",temp=1):
    
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    if prompt_type == "ZS":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_ZS, callbacks=[handler])
        summary = s2w_summarized[s2w_summarized["screenId"]==id]["summary"].iloc[0]
    elif prompt_type == "FS":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_FS, callbacks=[handler])
        summary = s2w_summarized[s2w_summarized["screenId"]==id]["summary"].iloc[0]
    else:
        return "ERROR: Unknown prompt type provided!"

    
    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    if prompt_type == "ZS":
        output_file = OUTPUT_FOLDER + f"/Zero_shot/s2w_summarized/{id}.html"
    elif prompt_type == "FS":
        output_file = OUTPUT_FOLDER + f"/Few_shot/s2w_summarized/{id}.html"

    with open(output_file, 'w') as f:
        f.write(response)


In [5]:
def creation_pipeline(prompt_type):
    screens = s2w_sample["screenId"].head(10).values
    for screen_id in screens:
        create_GUI(screen_id, prompt_type=prompt_type)
        print(f"{screen_id}: creation done")

In [6]:
creation_pipeline("FS")

33: creation done
70: creation done
300: creation done
486: creation done
494: creation done
495: creation done
498: creation done
549: creation done
596: creation done
761: creation done
